# Sistema de filtragem colaborativa

Este notebook tem como objetivo ser uma PoC (Proof of Concept) de um sistema de recomendação de item (baseado em filtragem colaborativa) em um dataset de filmes e ratings por usuario.

# Instalação

Este notebook utiliza a biblioteca *surprise* para a construção do modelo de recomendação. Esta biblioteca não existe por padrão no colab portanto devemos instalá-la através do pip.

Esta biblioteca faz parte do "guarda-chuva" das bibliotecas do scikit-learn, portanto não fugirá muito dos preceitos e princípios aprendidos em aula.

In [2]:
# Installing dependencies

!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 4.9 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633975 sha256=b4595b5c62241b80de197cf2b04f433255c73a5e7c473985ece713c3809024a2
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


# Importação de Dependências

Utilizaremos, além da biblioteca surprise, as bibliotecas *pandas* e *scikit-learn*, que já vêm instaladas por padrão no colab.

Além disso, realizei os imports das bases utilizadas e hospedadas no GitHub.

Link do conteúdo original dos datasets: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset

In [3]:
import pandas as pd
from surprise import KNNWithMeans, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD

MOVIES_METADATA_PATH = 'https://raw.githubusercontent.com/murilo-bracero/filtragem_colaborativa_unicsul/main/movies_metadata.csv'
MOVIES_RATINGS_PATH = 'https://raw.githubusercontent.com/murilo-bracero/filtragem_colaborativa_unicsul/main/ratings_small.csv'

# Extração dos dados crus

Neste momento utilizamos a biblioteca *pandas* para ler e indexar os conteúdos das bases

In [4]:
# Extraction

df_metadata = pd.read_csv(MOVIES_METADATA_PATH)
df_ratings = pd.read_csv(MOVIES_RATINGS_PATH)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Limpeza dos dados

Aqui realizo uma limpeza dos dados, tanto da base de classificações (*ratings*) quanto da de filmes propriamente ditos, removendo colunas que não utilizaremos

In [5]:
# Cleaning columns
df_ratings.drop('timestamp', axis=1, inplace=True)

df_metadata.drop(['belongs_to_collection', 'genres', 'overview'], axis=1, inplace=True)



## Removendo *outliers* e fusão dos datasets

Para facilitar a análise dos dados pelo modelo e futuramente pelo código do notebook, realizo aqui uma remoção de *outliers*, ou seja, dados malformados ou de baixa relevância para nosso modelo que, neste caso, somente atrapalharia a análise.

Além da limpeza, uno o dataset de ratings com o dataset de metadados dos filmes para facilitar a busca destes filmes no futuro.

In [6]:
# Cleaning data

# Removing outliers
df_metadata = df_metadata.drop(labels=[19730, 29503, 35587], axis=0)

# transforming type id into int for merge
df_metadata['id'] = df_metadata['id'].astype(int)

# merge into a unique dataset
df_movies = pd.merge(df_ratings, df_metadata, how='inner', left_on='movieId', right_on='id')

# Criação do dataset de análise

Com os datasets devidamente limpos e unificados, agora é realizada a extração dos dados que será usados somente para o treino, teste e evaluação do modelo, utilizando somente as colunas de identificadores de usuários, classificação que o usuário deu ao filme e o título do mesmo. 

In [7]:
# Checking dataset

df_movies_subset = df_movies[['userId', 'title', 'rating']]

# Performando a leitura

Diferente da maioria dos modelos nativos do Scikit-Learn, a Surprise possui uma classe específica para leitura dos datasets, e utilizaremos ela nesta parte.

O objetivo dela é limitar e aumentar a performance da leitura do modelo em tempo de treino/teste, e precisa necessariamente seguir o modelo do dataset de análisec criado anteriormente.

Além disso, é nessa classe que estabelecemos os limites de nossas classificações. Como o dataset utilizado utiliza o padrão comum de classificação de filmes (de 1 a 5 estrelas), podemos estabelecer este limite logo na instância do *Reader*.

Para mais informações, [acesse a documentação oficial](https://surprise.readthedocs.io/en/stable/reader.html)

In [8]:
# Starting model building

# reading dataset
# rating scale can be mannually adjusted because ratings will ever be grather than 1 and less than 5
sp_reader = Reader(rating_scale=(1, 5))
sp_model_data = Dataset.load_from_df(df_movies_subset, sp_reader) 

# Separando testes e treinos

Nesta etapada utilizamos a função do scikit-learn `train_test_split`, já utilizada nas aulas anteriormente, que tem como objetivo separar de forma aleatória amostras de dados baseados em uma *seed* e em um indicador de tamanho para que passamos ter bases de treino e teste mais confiáveis.

In [9]:
# splitting dataset for fit and test

SEED=10

fit_set, test_set = train_test_split(sp_model_data, 
                                     test_size=0.33, # Extracting exact 1/3 of dataaset for testing
                                     random_state=SEED 
                                     )

# Criação do Modelo

Nesta etapa instânciamos o modelo de recomendação que será utilizado neste notebook.

O KNNWithMeans é um modelo de inteligência artifical focado em recomendação de conteúdos que utiliza uma estratégia de filtragem colaborativa de conteúdos para realizar recomendações genéricas para um grupo de usuários em comum.

É amplamente utilizado tanto por plataformas de entretenimento quanto por e-commerces para recomendar conteúdos para grupos de usuários semelhantes.

A sigla KNN deriva de K-Nearest Neighbors, ou seja, K-enésemos vizinhos próximos, que faz uma referência ao objetivo do modelo.

In [10]:
# Creating a user_based K-Nearest Neighbors model with ratings mean

ub_model = KNNWithMeans(k=5, sim_options={'user_based': True})

ub_model

# Treino do modelo

Aqui utilizamos a base de treino criada anteriormente

In [11]:
# Training model

ub_model.fit(fit_set)

Computing the msd similarity matrix...
Done computing similarity matrix.


# Teste do Modelo

Aqui testamos o modelo e medimos a RMSE, ou seja, a raiz quadrática de erro média, que avalia a qualidade do nosso modelo para com as previsões feitas.

A RMSE nos mostra que, dada a amostra de classificações que temos dos filmes, nosso modelo possui uma margem de erro entre 0.95 e 0.97 "estrelas", o que, dado o tamanho de nosso dataset de classificações, é um score OK.

In [12]:
# Testing model
test_results = ub_model.test(test_set)

In [13]:
# Getting our mean error matrix

rmse = accuracy.rmse(test_results, verbose=True)

RMSE: 0.9709


# Avaliando o modelo

Com nosso modelo treinado e testado, podemos começar a obter previsões com base no perfil de usuário desejado.

O modelo KNNWithMeans, especificamente, funciona com um sistema baseado em indexes, ou seja, têm-se como entrada o index da avaliação do usuário que se deseja colher recomendações e, baseado nela, o modelo busca os *K* perfis que tiveram uma opinião semelhante a dele e recomenda filmes baseados nessa opinião. 

In [19]:
# At this point the model training and testing are finished, so this is how works

MOVIE_INDEX = 312
NUM_OF_RECOMMENDATIONS = 5

print('Análise Selecionada: ')
selected_movie = df_movies.iloc[MOVIE_INDEX]
print(f"\t-{selected_movie['title']} - ⭐ {selected_movie['rating']}" )

print('\n')

print('Filmes recomendados: ')

recommended_movie_names = []

recommendations_pivot = NUM_OF_RECOMMENDATIONS

while len(recommended_movie_names) < NUM_OF_RECOMMENDATIONS:
  rec_i = ub_model.get_neighbors(MOVIE_INDEX,recommendations_pivot)
  df_selected = df_movies_subset.iloc[rec_i]

  for record in df_selected.to_records():
    recommended_movie_names.append(record[2])

  recommended_movie_names = list(dict.fromkeys(recommended_movie_names))
  recommendations_pivot = recommendations_pivot + 1

for name in recommended_movie_names:
  print('\t-', name)

Análise Selecionada: 
	-The Dark - ⭐ 4.0


Filmes recomendados: 
	- Greed
	- American Pie
	- My Tutor
	- Jay and Silent Bob Strike Back
	- Confidentially Yours


# Conclusão

Através deste notebook podemos observar mais de perto o verdadeiro poder da inteligência artificial industrial, mais precisamente sua utilidade para com a recomendação de conteúdos para grupos de usuários.

Um outro motivo pelo qual este modelo é amplamente utilizado é que, como ele leva em consideração avaliações já realizadas por usuários, seu tempo de vida acaba sendo maior quando em comparação a modelos de recomendação pessoal em tempo real, que exigem massas de dados muito mais densas, detalhadas e tem um período útil relativamente curto até ter de ser treinado novamente.